In [1]:
import requests
import pandas as pd
from glob import glob

In [2]:
requests.post(url="http://127.0.0.1:8081/api/roleType/save", json={"name":"ROLE_USER"})
requests.post(url="http://127.0.0.1:8081/api/roleType/save", json={"name":"ROLE_ADMIN"})

<Response [201]>

In [3]:
category_list = ['상의', '바지', '스커트', '아우터', '반팔', '긴팔', '셔츠', '반바지', '슬랙스', '데님팬츠', '미니스커트', '롱스커트', '롱패딩', '숏패딩', '코트', '트렌치 코트']

category_save = "http://127.0.0.1:8081/main/category/save"

for category in category_list:
    request_body = {"catename":category}
    requests.post(url=category_save, json=request_body)

In [4]:
csv_file = glob("./상품DataFrame/*.csv")
csv_file

['./상품DataFrame\\바지_데님팬츠.csv',
 './상품DataFrame\\바지_반바지.csv',
 './상품DataFrame\\바지_슬랙스.csv',
 './상품DataFrame\\상의_긴팔.csv',
 './상품DataFrame\\상의_반팔.csv',
 './상품DataFrame\\상의_셔츠.csv',
 './상품DataFrame\\스커트_롱스커트.csv',
 './상품DataFrame\\스커트_미니스커트.csv',
 './상품DataFrame\\아우터_롱패딩.csv',
 './상품DataFrame\\아우터_숏패딩.csv',
 './상품DataFrame\\아우터_코트.csv',
 './상품DataFrame\\아우터_트렌치 코트.csv']

In [5]:
df = pd.read_csv(csv_file[0])
df = df.iloc[:, 1:]
df

,상품명,가격,이미지 경로,이미지 개수
0,시크릿밴드 롱 루즈 와이드 데님,"45,000원","['./상품이미지/바지/데님팬츠/시크릿밴드 롱 루즈 와이드 데님1.jpg', './...",3
1,딥워싱 브러쉬 루즈핏 와이드 스트레이트 데님,"46,800원",['./상품이미지/바지/데님팬츠/딥워싱 브러쉬 루즈핏 와이드 스트레이트 데님1.jp...,3
2,NOI024 스트레이트 루즈핏 데님 팬츠,"39,000원",['./상품이미지/바지/데님팬츠/NOI024 스트레이트 루즈핏 데님 팬츠1.jpg'],1
3,BUCKLE WIDE DENIM PANTS BLACK,"55,200원",['./상품이미지/바지/데님팬츠/BUCKLE WIDE DENIM PANTS BLAC...,3
4,가을용 커버밴드 이염없는 생지 와이드루즈 데님,"52,200원",['./상품이미지/바지/데님팬츠/가을용 커버밴드 이염없는 생지 와이드루즈 데님1.j...,3
...,...,...,...,...
85,LIBERTY WIDE PANTS_BLUE,"99,000원",['./상품이미지/바지/데님팬츠/LIBERTY WIDE PANTS_BLUE1.jpg'],1
86,Road jeans,"78,000원","['./상품이미지/바지/데님팬츠/Road jeans1.jpg', './상품이미지/바...",3
87,키르시 하이웨스트 코팅 데님 팬츠 G,"98,100원","['./상품이미지/바지/데님팬츠/키르시 하이웨스트 코팅 데님 팬츠20.jpg', '...",3
88,Mars jeans,"57,000원","['./상품이미지/바지/데님팬츠/Mars jeans1.jpg', './상품이미지/바...",3


이미지 먼저 순회하면서 저장한 후 아이템 저장하고,
아이템에 해당 이미지들과 카테고리들을 넣어줌.

In [6]:
item_save_url = 'http://127.0.0.1:8081/main/item/save'
image_save_url = 'http://127.0.0.1:8081/main/itemImage/save'
category_to_item = 'http://127.0.0.1:8081/main/category/add-to-item'
image_to_item = 'http://127.0.0.1:8081/main/itemImage/add-to-item'

for csv in csv_file:
    tmp = csv.split('\\')[1][:-4]
    upper, lower = tmp.split('_') # 카테고리
    df = pd.read_csv(csv)
    df = df.iloc[:, 1:]

    for i in range(90):
        name = df['상품명'][i]
        price = df['가격'][i][:-1].split(',') # '원' 제거하고 , split
        price = int(''.join(price))
        image_list = df['이미지 경로'][i][1:-1] # [, ] 제거
        images = []
        for image_path in image_list.split(','):
            images.append(image_path.strip()[1:-1]) # 공백 제거 후 ', ' 제거

        # 아이템 저장
        item_request_body = {
            "itemname":name,
            "price":price,
            "purchasecnt":"0",
            "count":"200",
            "reviewmean":"0",
            "reviewcount":"0"}
        requests.post(url=item_save_url, json=item_request_body)

        # 이미지 저장
        for img in images:
            image_request_body = {
                'imgpath':img}
            requests.post(url=image_save_url, json=image_request_body)

        # Category To Item
        for cate in [upper, lower]:
            cate_item_body = {
                "itemName":name,
                "categoryName":cate}
            requests.post(url=category_to_item, json=cate_item_body)

        # Image to Item
        for img in images:
            image_request_body = {
                "itemName":name,
                "imgPath":img}
            requests.post(url=image_to_item, json=image_request_body)